In [1]:
import earthaccess

In [2]:
earthaccess.login()

Enter your Earthdata Login username:  aimeeb
Enter your Earthdata password:  ········


In [10]:
s3fs = earthaccess.get_s3_filesystem(daac="ORNLDAAC")

In [4]:
import xarray as xr

In [15]:
#url = 's3://prod-lads/VNP46A1/VNP46A1.A2016156.h18v03.002.2024236145511.h5'
url = 's3://ornl-cumulus-prod-protected/daymet/Daymet_Daily_V4R1/data/daymet_v4_daily_na_dayl_2004.nc'
ds = xr.open_dataset(s3fs.open(url), engine="h5netcdf", decode_times=False)

In [16]:
list(ds.data_vars.keys())

['yearday', 'time_bnds', 'lambert_conformal_conic', 'dayl']